In [19]:
import os
import scanpy as sc
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

# Initialize constants
load_dotenv()
CONDITION_KEY, CELL_TYPE_KEY = os.getenv('CONDITION_KEY'), os.getenv('CELL_TYPE_KEY')

from load_data import get_adata
from sc_condition_prediction import create_and_train_vae_model, evaluate_r2, N_INPUT, N_LAYERS, N_HIDDEN, N_LATENT, BATCH_SIZE
# from utils import remove_stimulated_for_celltype

# Load data
# train_adata = get_adata(verbose=True)
# train_new = remove_stimulated_for_celltype(train_adata, celltype="CD4T")

control


KeyError: 'controlled'

In [ ]:
train_adata.obs

In [30]:
train_adata_by_cell_type = {}

cell_types = train_adata.obs.cell_type.cat.categories.values

for cell_type in cell_types:
    train_adata_by_cell_type[cell_type] = train_adata[train_adata.obs.cell_type == cell_type]

In [31]:
train_adata_by_cell_type

{'CD4T': View of AnnData object with n_obs × n_vars = 5564 × 6998
     obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type'
     var: 'gene_symbol', 'n_cells'
     uns: 'cell_type_colors', 'condition_colors', 'neighbors'
     obsm: 'X_pca', 'X_tsne', 'X_umap'
     obsp: 'distances', 'connectivities',
 'CD14+Mono': View of AnnData object with n_obs × n_vars = 2561 × 6998
     obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type'
     var: 'gene_symbol', 'n_cells'
     uns: 'cell_type_colors', 'condition_colors', 'neighbors'
     obsm: 'X_pca', 'X_tsne', 'X_umap'
     obsp: 'distances', 'connectivities',
 'B': View of AnnData object with n_obs × n_vars = 1811 × 6998
     obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type'
     var: 'gene_symbol', 'n_cells'
     uns: 'cell_type_colors', 'condition_colors', 'neighbors'
     obsm: 'X_pca', 'X_tsne', 'X_umap'
     obsp: 'distances', 'connectivities',
 'CD8T': View of AnnData object with n_obs × n_vars = 1115 

In [32]:
cd4t_df = train_adata_by_cell_type['CD4T'].to_df()
cd4t_df.describe()

index,AL627309.1,RP11-206L10.9,LINC00115,NOC2L,KLHL17,HES4,ISG15,TNFRSF18,TNFRSF4,SDF4,...,C21orf67,FAM207A,ADARB1,POFUT2,COL18A1,SLC19A1,COL6A2,FTCD,DIP2A,S100B
count,5564.000000,5564.000000,5564.000000,5564.000000,5564.000000,5564.000000,5564.000000,5564.000000,5564.000000,5564.000000,...,5564.000000,5564.000000,5564.000000,5564.000000,5564.000000,5564.000000,5564.000000,5564.000000,5564.000000,5564.000000
mean,0.000290,0.000295,0.004717,0.081097,0.000599,0.024535,1.396227,0.040431,0.091370,0.064878,...,0.000406,0.043993,0.004070,0.009166,0.009337,0.004598,0.001317,0.000346,0.020588,0.021828
std,0.015311,0.015755,0.063308,0.254489,0.022557,0.154765,1.192688,0.189054,0.297386,0.230153,...,0.022359,0.187557,0.057041,0.087452,0.087252,0.064468,0.036036,0.018251,0.131128,0.152459
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.626387,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.427323,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.823251,0.953000,1.159003,2.176398,0.927066,2.231160,5.175874,1.898126,2.472356,2.879868,...,1.468089,1.580605,1.017806,1.538911,1.213284,1.514670,1.494599,0.996147,1.607041,2.438123
